In [ ]:
%pip install langchain python-dotenv

In [2]:
# LangChain
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate

# Environment Variables
import os

# Loading of environment variables (API keys, etc.)
from dotenv import load_dotenv

# LinkedIn requirements
import requests
import json

load_dotenv()

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')

In [5]:
llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key, model_name='gpt-4', client="my_client")


In [25]:
def get_users_posts(screen_name, posts_to_return=12):
    url = "https://fresh-linkedin-profile-data.p.rapidapi.com/get-profile-posts"

    querystring = {"linkedin_url": f"https://www.linkedin.com/in/{screen_name}/", "type": "posts"}   # replace with your LinkedIn URL
    
    headers = {
	"X-RapidAPI-Key": "abc123", # replace with your API key
	"X-RapidAPI-Host": "somehost.com" # replace with your host
    }

    response = requests.get(url, headers=headers, params=querystring)

    # load the data into a Python dictionary
    data = json.loads(response.text)
    print(response.text)

    # access the 'data' field (a list of posts)
    posts = data['data']

    # sort posts by 'num_likes' in descending order
    sorted_posts = sorted(posts, key=lambda post: post['num_likes'], reverse=True)

    # get the 'text' field of the top 100 posts
    texts = [post['text'] for post in sorted_posts[:posts_to_return]]   

    # join the texts into a single string separated by two new lines
    example_posts = '\n\n'.join(texts)
    return example_posts

In [ ]:
user_screen_name = 'sean-fay-mba-8a598b9'  # Replace this with the desired user's screen name
users_tweets = get_users_posts(user_screen_name)

In [ ]:
print(example_posts)

In [ ]:
template = """
Please create me a LinkedIn post about going to the park and eating a sandwich.

% TONE
 - Don't use any emojis or hashtags.
 - Respond in the tone of Steve Chambers

% START OF EXAMPLE POSTS TO MIMIC
{example_posts}
% END OF EXAMPLE POSTS TO MIMIC

YOUR POST:
"""

prompt = PromptTemplate(
    input_variables=["example_posts"],
    template=template,
)

final_prompt = prompt.format(example_posts=example_posts)

print (final_prompt)

In [ ]:
class ChatOpenAI:
    def predict(self, prompt):
        # implementation of the predict method
        return "prediction"
print (output)

In [ ]:
prompt = """
Can you please generate a list of tone attributes and a description to describe a piece of writing by?

Things like pace, mood, etc.

Respond with nothing else besides the list
"""

how_to_describe_tone = llm.predict(prompt)
print (how_to_describe_tone)

In [13]:
def get_authors_tone_description(how_to_describe_tone, sorted_posts):
    template = """
        You are an AI Bot that is very good at generating writing in a similar tone as examples.
        Be opinionated and have an active voice.
        Take a strong stance with your response.

        % HOW TO DESCRIBE TONE
        {how_to_describe_tone}

        % START OF EXAMPLES
        {sorted_posts}
        % END OF EXAMPLES

        List out the tone qualities of the examples above
        """

    prompt = PromptTemplate(
        input_variables=["how_to_describe_tone", "sorted_posts"],
        template=template,
    )

    final_prompt = prompt.format(how_to_describe_tone=how_to_describe_tone, sorted_posts=sorted_posts)

    authors_tone_description = llm.predict(final_prompt)

    return authors_tone_description

In [ ]:
authors_tone_description = get_authors_tone_description(how_to_describe_tone, sorted_posts)
print (authors_tone_description)

In [14]:
template = """
% INSTRUCTIONS
 - You are an AI Bot that is very good at mimicking an author writing style.
 - Your goal is to write content with the tone that is described below.
 - Do not go outside the tone instructions below
 - Do not use hashtags or emojis
 - Respond in the tone of Steve Chambers

% Description of the authors tone:
{authors_tone_description}

% Authors writing samples
{sorted_posts}

% YOUR TASK
Please create a LinkedIn about going to the park and eating a sandwich.
"""

prompt = PromptTemplate(
    input_variables=["authors_tone_description", "sorted_posts"],
    template=template,
)

final_prompt = prompt.format(authors_tone_description=authors_tone_description, sorted_posts=sorted_posts)

In [ ]:
llm.predict(final_prompt)